# Welcome to GPT Tokenizer

😔

We have a sad face because, `Tokenizers` are the least favourite part of **Large Language Models (LLMs)** that I need to work with. But unfortunately it is completely necessary to understand in detail to work with them... And a lot of oddness with LLMs traces back the these `Tokenization`...

So...

What is `Tokenization`?

Now in our last notebook (<a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20from%20Scratch.ipynb">GPT from Scratch</a>) we already implemented tokenization but it was done in a completely naïve and simple way...

In the previous notebook, the question we encountered was, "how do we *plug-in* text into the GPT?" and we ended up with a vocabulary of `92` characters, from which we created two look-up tables `stoi` and `itos` for mapping characters to indeces and vice-versa, which could be used as a `token` table for encode and decode functions, where `encode` function returned the encoded token integers and `decode` function returned the decoded message from the encoded tokens...

And later we saw that the way we *plug* these `tokens` into the model is by using a `tokenEmbeddingTable` where this table represented a row of `92` possible characters with their respective `embeddings`...

But,

In practice, people use a lot more complecated schemes to encode and decode these `tokens`...

And we deal with **chunk level** texts. And these **chunk level** texts are constructed using algorithms like `Byte-Pair` algorithms which we will be covering in a bit...

I'd also like discuss the paper that introduced this concept of `byte` level tokenization encoding as a mechanism in the context of LLMs...

Which is this paper : <a href="https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf">Language Models are Unsupervised Multitask Learners</a>

And if we scroll to the point **2.2 Input Representation** within the paper we see that they conclude with the line: "The vocabulary is expanded to 50,257. We also increase the context size from 512 to 1024 tokens an a larger batchsize of 512 is used."

Which means that the `vocabularySize` they used is about `50,257` and in the `Transformer` architecture's attention layer, every single token is *attending* to previous tokens in a sequence, and it is able to see upto `1024 tokens` in a sequence...

So **`Tokens` are the fundamental atomic units of a LLMs.** and everything related to it...

And **`Tokenization` is the process of translating `strings` or text into sequences of `tokens` and vice versa.**

And we can also look into the <a href="https://arxiv.org/pdf/2307.09288">Llama 2: Open Foundation and Fine-Tuned Chat Models</a> paper by Meta and we see that, in their paper in section **2.1** they mentioned that they trained their model with 2 trillion tokens of data...

And luckily the `Byte-Pair` algorithm is fairly simple and we can implement it ourselves and we can build our own `tokenizer`..

# Tiktokenizer

Before we dive into the code we can go to this nice website that has been created for us <a href="https://tiktokenizer.vercel.app/">Tiktokenizer</a> and familize ourselves with the tokenization types that are used ...

And what's great about this website is, tokenization is runs live on our browsers with the help of JavaScript... So we can plug in our own text and test the different tokenization techniques and their respective outputs and color coded tokens...

Now what I'd like you to do is plug this sample text into the website input:

```text
Today we will be learning tokenization. It is not very fun, but definitely informative.

69 + 420 = 489
6969 + 420 = 7389

Hat.
hat.
HAT.
I have a hat.

私はコンピューターが大好きです。私は機械学習エンジニアです。 

for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
```

And set the tokenizer to `GPT-2` because that's what we discussed earlier to relate what's happening currently...

And we immediately see that we get this kind of output:
![Tokenizer_GPT2Test](ExplanationMedia/Images/Tokenizer_GPT2Test.png)